In [2]:
import cpp_soft_info

# Load Data

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
from result_saver import SaverProvider
provider = SaverProvider()

In [5]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibmq_mumbai'
DISTANCE = 11
ROUNDS = 11

md = metadata_loader(_extract=True, _drop_inutile=False)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["distance"] == DISTANCE]
md = md[md["backend_name"]==DEVICE]
md = md[md["rounds"]==ROUNDS]


md = md[md["meas_level"] == 1]
md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,job_name,job_metadata,tags,meas_level,init_qubits,meas_return,...,job_status,execution_date,extra,optimization_level,code,distance,rounds,logical,layout,descr
2580,2023-12-12 16:41:47.352000+01:00,mumbai_RepCodes,ibmq_mumbai,cnw7yes8ybr000847p7g,NaN,{},[RepCode_Count_job],1.0,True,single,...,JobStatus.DONE,2023-12-12 16:48:26.705057+01:00,NaN,NaN,RepetitionCodeCircuit,11.0,11,1,_is_hex=True,Run bigger Repetition codes v4: new distances ...
2579,2023-12-12 16:41:44.776000+01:00,mumbai_RepCodes,ibmq_mumbai,cnw7ye18ybr000847p70,NaN,{},[RepCode_Count_job],1.0,True,single,...,JobStatus.DONE,2023-12-12 16:48:12.274652+01:00,NaN,NaN,RepetitionCodeCircuit,11.0,11,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...


In [6]:
memories = {}
jobs = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()
    jobs[mmr_name] = job_id

memory_str = "mmr_log_0"
memory = memories[memory_str][:]
job_id = jobs[memory_str]
print(memory.shape)

(8264, 121)


# Initialize the code

In [7]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid


layout = get_repcode_layout(distance=DISTANCE, backend=provider.get_backend(DEVICE), _is_hex=False)
qubit_mapping = get_repcode_IQ_map(layout, ROUNDS) #Hardcoded for repetition codes
print("layout:", layout)
print("len(layout):", len(layout), " = distance * code qubits + (distance-1) * link qubits")

Finding the longest path starting from 27 qubits: 100%|██████████| 27/27 [00:00<00:00, 20150.57it/s]

layout: [1, 3, 8, 14, 19, 25, 23, 18, 12, 7, 0, 2, 5, 11, 16, 22, 24, 21, 15, 10, 4]
len(layout): 21  = distance * code qubits + (distance-1) * link qubits


In [8]:
other_date = None # if none then it will find the closest to the tobecalib_job date
other_date = "2023-11-10T10:30:00" # "2023-11-22T" works to
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, job_id, 40, 0.8, other_date=other_date)
# grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, job_id, 300, 2, other_date=other_date)

# takes 10s

Specified job execution date: 2023-12-12 16:48:12.274652+01:00
Found jobs for backend ibmq_mumbai with closest execution date 2023-11-21 18:54:05.610765+00:00. Retrieving kde grid...
Searching for ibmq_mumbai and 23.11.21_10h17_40pts_0.8std


#### Noise model stim

In [9]:
from datetime import datetime

def get_avgs_from_dict(noise_dict):
    '''
    Takes noise dictionary as an argument then returns length 5
    array defining average noise levels: [init, idle, RO, single-, two-qubit gate]
    '''
    init_avg = np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    idle_avg = np.average([noise_dict[qubit]['idle'] for qubit in noise_dict])
    RO_avg = np.average([noise_dict[qubit]['RO'] for qubit in noise_dict])
    single_avg = 1e-4#np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    two_gate_avg = np.average([noise_dict[qubit]['2-gate'][connection]
                        for qubit in noise_dict
                        for connection in noise_dict[qubit]['2-gate']
                        if connection != 'default'])
    return [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]
# End get_avgs_from_dict


def get_noise_dict_from_backend(backend, layout, date=None):
    '''
    Takes a given backend and a mapping object and returns a noise dictionary
    to pass to a new Cross_Platform_Code/Hex_Code to have accurate backend
    specific noise information based on calibration data.
    '''
    # Pre-reqs
    noise_dict = {}
    round_time = 1000e-9
    if date is None:
        t = None
    else:
        t = datetime(day=int(date[-2:]), month=int(date[-4:-2]), year=int(date[:4]))
    properties = backend.properties(datetime=t)
    # Initializing each qubit with single qubit noise
    for qubit in layout:
        # Defining ROI error
        ROI_error = properties.readout_error(qubit)
        # Defining idle error
        t1 = properties.t1(qubit)
        t2 = properties.t2(qubit)
        t_time = min(t1, t2)
        idle_error = 1 - np.exp(-round_time / t_time)
        # Updating dictionary
        noise_dict[qubit] = {}
        noise_dict[qubit]['init'] = ROI_error
        noise_dict[qubit]['idle'] = idle_error
        noise_dict[qubit]['RO'] = ROI_error
        noise_dict[qubit]['gate'] = 1e-4
        noise_dict[qubit]['2-gate'] = {'default': 1e-2}
    # Two qubit gate noise
    all_qiskit_indexes = layout 
    for pair in backend.coupling_map:
        # Making sure connection is in circuit
        if pair[0] in all_qiskit_indexes and pair[1] in all_qiskit_indexes:
            # Getting two qubit gate error
            two_gate_error = properties.gate_error('cx', pair)
            if two_gate_error > .5:
                two_gate_error = .5
            # # Updating dictionary
            # stim_index_0 = map.get_stim_index(qiskit_index=pair[0])
            # stim_index_1 = map.get_stim_index(qiskit_index=pair[1])
            noise_dict[pair[0]]['2-gate'][pair[1]] = two_gate_error
            noise_dict[pair[1]]['2-gate'][pair[0]] = two_gate_error
    return noise_dict
# End get_noise_dict_from_backend

In [10]:

date = "2023-10-30 09:45:17.340386+01:00" #md.creation_date[0] # md[0] for log 1
noise_dict = get_noise_dict_from_backend(provider.get_backend(DEVICE), layout, date=None) # takes noise closest to today
print("noise_dict:", noise_dict)
avgs_noise = get_avgs_from_dict(noise_dict)
print("avgs_noise [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]:", avgs_noise)

# takes 4s

noise_dict: {1: {'init': 0.011700000000000044, 'idle': 0.009264085822753843, 'RO': 0.011700000000000044, 'gate': 0.0001, '2-gate': {'default': 0.01, 0: 0.005124950217809426, 2: 0.009420430723780487, 4: 0.005358785229142682}}, 3: {'init': 0.014100000000000001, 'idle': 0.012687261250991178, 'RO': 0.014100000000000001, 'gate': 0.0001, '2-gate': {'default': 0.01, 2: 0.004997507692207137, 5: 0.038329938870632824}}, 8: {'init': 0.006666666666666599, 'idle': 0.005745540943499883, 'RO': 0.006666666666666599, 'gate': 0.0001, '2-gate': {'default': 0.01, 5: 0.5, 11: 0.005748291312218018}}, 14: {'init': 0.031400000000000095, 'idle': 0.007052368426888056, 'RO': 0.031400000000000095, 'gate': 0.0001, '2-gate': {'default': 0.01, 11: 0.005821505834515434, 16: 0.01064218449205348}}, 19: {'init': 0.1259, 'idle': 0.008772507651417127, 'RO': 0.1259, 'gate': 0.0001, '2-gate': {'default': 0.01, 16: 0.026454597748886532, 22: 0.012925592773663425}}, 25: {'init': 0.01550000000000007, 'idle': 0.00595711167654977

In [11]:
from soft_info import draw_matching_graph



circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization= avgs_noise[4], #two-qubit-fidelity,
                                after_reset_flip_probability= 0, #reset error,
                                before_measure_flip_probability= avgs_noise[2], #measurement error,
                                before_round_data_depolarization= 0)#avgs_noise[1]) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)
# draw_matching_graph(matching, d, synd_rounds, figsize=(20, 20))   
# takes 10s

# Get weights

In [12]:
def weight_to_prob(weight):
    return 1/(1+np.exp(weight))

print(weight_to_prob(3.6))

0.026596993576865856


In [13]:
from tqdm import tqdm

means = []
stds = []
means_prob = []
stds_prob = []
for shot in tqdm(memory):
    cpp_soft_info.soft_reweight_pymatching(matching._matching_graph, [shot], ROUNDS,
                                        qubit_mapping, grid_dict, processed_scaler_dict,
                                        p_data=-1, p_mixed=-1, common_measure=-1)
    time_weights = []
    time_probs = []
    for edge in matching.edges():
        node1 = edge[0]
        node2 = edge[1]
        weight = edge[2]['weight']
        if node2 is not None:
            if node2 == node1 + (DISTANCE-1):
                time_weights.append(weight)
                time_probs.append(weight_to_prob(weight))

    means.append(np.mean(time_weights))
    stds.append(np.std(time_weights))
    means_prob.append(np.mean(time_probs))
    stds_prob.append(np.std(time_probs))

print("means:", means)
print("stds:", stds)
print("mean of the means:", np.mean(means), "for", len(means), "shots")
print("mean of the stds:", np.mean(stds), "for", len(stds), "shots")

print()
print("means_prob:", means_prob)
print("stds_prob:", stds_prob)
print("mean of the means_prob:", np.mean(means_prob), "for", len(means_prob), "shots")
print("mean of the stds_prob:", np.mean(stds_prob), "for", len(stds_prob), "shots")

100%|██████████| 8264/8264 [00:05<00:00, 1443.80it/s]

means: [3.533267597480291, 3.2849015223022335, 3.8409797101876637, 3.5831634597352733, 3.4006096352652397, 3.8241734369754012, 3.3062870240905977, 3.566070890443043, 3.1351816228866376, 3.36215049492215, 3.293952116803694, 3.2672962289097724, 3.425789621038631, 3.772128860630287, 3.3298298732706284, 3.170506251034452, 3.563917943842996, 3.3559485339942703, 3.178173599377602, 3.1401623037082023, 3.4683800221492556, 3.552161914656557, 3.3141870553386297, 3.4038479525563496, 3.1099004388226965, 3.6632844955706516, 3.607415712113674, 3.338463102371728, 3.753859679719856, 3.370267334993255, 3.31314082475098, 3.3518200951219335, 3.3583187067521334, 3.724010196851348, 3.263137270040629, 3.1969133623071753, 3.3676547991758734, 3.4968267200349024, 3.2268862035625294, 3.209618142849935, 3.4054838975116395, 3.4563589190376014, 3.567678626419412, 3.4787197836188786, 3.4556748998802034, 3.3439555319087124, 3.419228990397465, 3.339287350939132, 3.1945544877798207, 3.2933904332249293, 3.8945835694209

In [14]:
cpp_soft_info.soft_reweight_pymatching(matching._matching_graph, memory[20:], ROUNDS,
                                        qubit_mapping, grid_dict, processed_scaler_dict,
                                        p_data=-1, p_mixed=-1, common_measure=-1)

print(matching.edges())
time_weights = []
time_probs = []
for edge in matching.edges():
    node1 = edge[0]
    node2 = edge[1] 
    weight = edge[2]['weight']
    if node2 is not None:
        if node2 == node1 + (DISTANCE-1):
            time_weights.append(weight)
            time_probs.append(weight_to_prob(weight))

print("time_weights:", time_weights)
print("min(time_weights):", min(time_weights))
print("max(time_weights):", max(time_weights))
print("variance(time_weights):", np.var(time_weights))
print("mean(time_weights):", np.mean(time_weights))
print("std(time_weights):", np.std(time_weights))

print() 

print("time_probs:", time_probs)
print("min(time_probs):", min(time_probs))
print("max(time_probs):", max(time_probs))
print("std(time_probs):", np.std(time_probs))
print("mean(time_probs):", np.mean(time_probs))

[(0, None, {'fault_ids': set(), 'weight': 4.007074205069798, 'error_probability': 0.017861685487136927}), (0, 1, {'fault_ids': set(), 'weight': 4.700138687926212, 'error_probability': 0.009012059968443282}), (0, 10, {'fault_ids': set(), 'weight': 3.6270943111505574, 'error_probability': 0.04115696246771565}), (0, 11, {'fault_ids': set(), 'weight': 4.007074205069797, 'error_probability': 0.01786168548713693}), (1, 2, {'fault_ids': set(), 'weight': 4.700138687926212, 'error_probability': 0.009012059968443282}), (1, 11, {'fault_ids': set(), 'weight': 3.7062800518569237, 'error_probability': 0.0494272044084029}), (1, 12, {'fault_ids': set(), 'weight': 4.007074205069797, 'error_probability': 0.01786168548713693}), (2, 3, {'fault_ids': set(), 'weight': 4.700138687926212, 'error_probability': 0.009012059968443282}), (2, 12, {'fault_ids': set(), 'weight': 1.149548994568745, 'error_probability': 0.0494272044084029}), (2, 13, {'fault_ids': set(), 'weight': 4.007074205069797, 'error_probability':

# Decode

In [15]:
matching = pymatching.Matching.from_detector_error_model(model)
# cpp_soft_info.reweight_edges_based_on_error_probs(matching._matching_graph, counts, False, "spitz")

p_data = 6.869e-3 # mean sherbrooke noise
num_errors = cpp_soft_info.decode_IQ_shots(matching._matching_graph, memory,
                                           ROUNDS, qubit_mapping, grid_dict,
                                           processed_scaler_dict, p_data=-1, p_mixed=-1, #p_mixed=1e-80, for d=30
                                           common_measure=-1, _bimodal=False, merge_strategy = "replace")
print("num_errors:", num_errors, "out of", len(memory), "shots")
 
# takes 5s

num_errors: 11 out of 8264 shots


In [16]:
matching = pymatching.Matching.from_detector_error_model(model)
num_errors = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, memory,
                                           ROUNDS, qubit_mapping, grid_dict,
                                           processed_scaler_dict)
print("num_errors:", num_errors, "out of", len(memory), "shots")

# takes 1s

num_errors: 27 out of 8264 shots


In [18]:
matching = pymatching.Matching.from_detector_error_model(model)

def weight_to_prob(weight):
    return 1/(1+np.exp(weight))

p_data = 6.869e-3 # mean sherbrooke ECR error
p_mixed = p_data/1 # Same as weighted
# p_meas = 1e-3
# p_meas = 15.900e-2 # random found number
p_meas = weight_to_prob(np.mean(means))
print("p_meas:", p_meas)
# p_meas = np.mean(means_prob)


p_data = -1
p_mixed = -1
# p_meas = -1

num_errors = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, memory, 
                                           ROUNDS, qubit_mapping, grid_dict, processed_scaler_dict,
                                           p_data, p_mixed, p_meas, common_measure=-1)

print("num_errors:", num_errors, "out of", len(memory), "shots")
         
# takes 1s

p_meas: 0.032324420624868025
num_errors: 18 out of 8264 shots
